# Utility functions (from P. Norvig)

In [1]:
# Python 3.x
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = './inputs/input{}'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        return urllib.request.urlopen("http://adventofcode.com/2017/day/{}/input".format(day))

def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

cat = ''.join

empty   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

Tests

In [4]:
assert tuple(transpose(((1, 2, 3), (4, 5, 6)))) == ((1, 4), (2, 5), (3, 6))
assert first('abc') == first(['a', 'b', 'c']) == 'a'
assert cat(['a', 'b', 'c']) == 'abc'
assert (groupby(['test', 'one', 'two', 'three', 'four'], key=len) 
        == {3: ['one', 'two'], 4: ['test', 'four'], 5: ['three']})

# Day 1, challenge 1

Need to obtain summation of all digits in a number such that the next digit is the same (circular)

In [31]:
f = Input(1)
data = f.read()[:-1]
f.close()

In [34]:
suma = 0
for i in range(len(data[:-1])):
    if data[i] == data[i+1]:
        suma = suma+int(data[i])
if data[0] == data[-1]:
    suma = suma+int(data[-1])

In [35]:
suma

1182

## Challenge 2
In this case we obtain summation of all digits such that the digit `len(data)/2` ahead (modulo `len(data)`) is equal to that one. 

In [38]:
suma = 0
for i in range(len(data)):
    j = (i+len(data)//2)%len(data)
    if data[i] == data[j]:
        suma = suma+int(data[i])
suma

1152

# Day 2
## Challenge 1
Find checksum of matrix

In [22]:
f = Input(2)
data = f.read()
data = data.split('\n')[:-1]
data = [d.split('\t') for d in data]
data = [[int(j) for j in i] for i in data]
f.close()

In [23]:
chcksm = 0
for line in data:
    chcksm += max(line) - min(line)
print(chcksm)

41919


## Challenge 2

In [25]:
chcksm = 0
for line in data:
    for j1 in range(len(i)): 
        for j2 in range(len(i)):
            if j2 != j1:
                if i[j1] % i[j2] == 0:
                    chcksm += int(line[j1]/line[j2])
                    break
print(chcksm)

303


# Day 3
## Challenge 1
Objective: Find coordinates of n-th element of a spiral defined as:
    
    17  16  15  14  13
    18   5   4   3  12
    19   6   1   2  11
    20   7   8   9  10
    21  22  23---> ...

In [33]:
def nextpos(num, prevpos):  # bruteforce ftw
    if num == 1:
        return np.array([0,0])
    if num == 2:
        return np.array([1,0])
    if prevpos[0] > abs(prevpos[1]):
        return prevpos + np.array([0, 1])
    elif abs(prevpos[0]) < prevpos[1] or (prevpos[0] == prevpos[1] and prevpos[0]>0):
        return prevpos + np.array([-1, 0])
    elif abs(prevpos[0]) > abs(prevpos[1]) or (prevpos[0] == -1*prevpos[1] and prevpos[1]>0):
        return prevpos + np.array([0, -1])
    else:
        return prevpos + np.array([1, 0])

def pos(num):
    if num == 1:
        return np.array([0,0])
    if num == 2:
        return np.array([1,0])
    prevpos = pos(num-1)
    if prevpos[0] > abs(prevpos[1]):
        return prevpos + np.array([0, 1])
    elif abs(prevpos[0]) < prevpos[1] or (prevpos[0] == prevpos[1] and prevpos[0]>0):
        return prevpos + np.array([-1, 0])
    elif abs(prevpos[0]) > abs(prevpos[1]) or (prevpos[0] == -1*prevpos[1] and prevpos[1]>0):
        return prevpos + np.array([0, -1])
    else:
        return prevpos + np.array([1, 0])

prevpos = np.array([0,0])
for i in range(1, 347991):
    prevpos = nextpos(i+1, prevpos)

In [35]:
cityblock_distance(prevpos)

480

## Challenge 2
Filling each of the coordinate with the sum of all adjacent coordinates (going in spiral and starting with a 1 on `[0,0]`), find what's the value of the first position with a value greater than `347991` (my puzzle input)

In [30]:
poss = {tuple(np.array([0,0])):1}
prevpos = np.array([0,0])
for i in range(1,347991):
    prevpos = nextpos(i+1, prevpos)
    num = 0
    for neig in neighbors8(prevpos):  # these are tuples already, fixed a typo in Norvig's functions.
        if neig in poss:
            num += poss[neig]
    poss[tuple(prevpos)] = num
    if num > 347991:
        print(num)
        break

349975


# Day 4
## Challenge 1
Given a list of strings find duplicates.

In [18]:
f = Input(4)
data = f.read()
data = data.split('\n')[:-1]
data = [d.split(' ') for d in data]
f.close()

In [19]:
res = [any(l.count(x)>1 for x in l) for l in data]
len(res) - sum(res)

451

## Challenge 2
Look for anagrams (need to sort strings!)

In [21]:
data2 = [[cat(sorted(list(x))) for x in l] for l in data]  # ''.join() = cat is not necessary, but I like it.
res = [len(l) > len(set(l)) for l in data2]  # Same as any(l.count(x)>1 for x in l) for l in data but can be faster.
len(res) - sum(res)

223

# Day 5
## Challenge 1

We traverse a sequence $(i_0(0), ..., i_0(n-1)$ the following way:
\begin{align}
&x_0 &= &0\\
&x_{t+1} &= &x_t + i_t(x_t)\\
&i_{t+1}(x_t) &= &i_{t}(x_t) + 1\\
\end{align}

Stop if $x_{t+1} > n-1$

In [36]:
f = Input(5)
data = f.read()
data = data.split('\n')[:-1]
data = [int(i) for i in data]
f.close()

In [37]:
i = 0
nexti = 0
step = 0
while i < len(data):
    nexti = i + data[i]
    data[i] = data[i] + 1
    i = nexti
    step = step+1
print(step)


354121


## Challenge 2

Now $i_{t+1}(x_t) = i_t(x_t) + 1$ if $i_t(x_t) < 3$ and $i_t(x_t) - 1$ else.

In [38]:
f = Input(5)
data = f.read()
data = data.split('\n')[:-1]
data = [int(i) for i in data]
f.close()

In [17]:
i = 0
nexti = 0
step = 0
while i < len(data):
    nexti = i + data[i]
    if data[i] >= 3:
        data[i] = data[i] - 1
    else:
        data[i] = data[i] + 1
    i = nexti
    step = step+1
print(step)


27283023


# Day 6
## Challenge 1 and 2

In [15]:
f = Input(6)
data = f.read().split('\t')
data = [int(i) for i in data]
f.close()
data

[4, 10, 4, 1, 8, 4, 9, 14, 5, 1, 14, 15, 0, 15, 3, 5]

In [16]:
def redist(inp):
    i = 0
    dat = 0
    for j in range(len(inp)):
        if inp[j] > dat:
            i = j
            dat = inp[j]
    inp[i] = 0
    while dat > 0:
        i = (i+1)%len(inp)
        inp[i] += 1
        dat -= 1
    return inp

mydict = {}
mydict[tuple(data)] = 0
step = 0
while True:
    step += 1
    redist(data)
    if tuple(data) in mydict:
        break
    else:
        mydict[tuple(data)] = step
print(step)
print(step-mydict[tuple(data)])
print(data)

12841
8038
[1, 0, 14, 14, 12, 11, 10, 9, 9, 7, 5, 5, 4, 3, 7, 1]


# Day 7
## Challenge 1

In [44]:
from anytree import Node, RenderTree, PostOrderIter

In [57]:
f = Input(7)
data = f.read().split('\n')[:-1]
data = [d.split(' -> ') for d in data]
nodedict = {}
for d in data:
    daux = d[0].split(' ')
    nodedict[daux[0]] = Node(daux[0], weight=int(daux[1][1:-1]))
for d in data:
    daux = d[0].split(' ')
    if len(d) > 1:
        for program in d[1].split(', '):
            nodedict[program].parent = nodedict[daux[0]]
f.close()

In [58]:
print(nodedict['czlmv'].root)
root = nodedict['aapssr']
print(root.depth)
len(root.children)

Node('/aapssr', weight=72)
0


5

## Challenge 2

In [59]:
def BalancedSubtowers(node):
    if len(node.children) == 0:
        return [0]
    weights = [child.weight + sum(BalancedSubtowers(child)) for child in node.children]
    return weights

for node in PostOrderIter(root):
    if len(set(BalancedSubtowers(node))) > 1:
        print(node)
        break

print(BalancedSubtowers(nodedict['pkowhq']))
print(nodedict['pkowhq'].children)

Node('/aapssr/qwada/pkowhq', weight=1187)
[1486, 1492, 1486, 1486]
(Node('/aapssr/qwada/pkowhq/zfrsmm', weight=763), Node('/aapssr/qwada/pkowhq/tlskukk', weight=1464), Node('/aapssr/qwada/pkowhq/fqkbscn', weight=418), Node('/aapssr/qwada/pkowhq/mlafk', weight=38))


# Day 8
## Challenge 1

In [2]:
from collections import defaultdict

In [12]:
f = Input(8)
data = f.read().split('\n')[:-1]
f.close()

In [6]:
regs = defaultdict(lambda: 0)
def parse(line):
    laux = line.split(' ')
    cond = laux[-3:]
    cond[0] = "regs['{}']".format(cond[0])
    if eval(' '.join(cond)):
        if laux[1] == 'inc':
            regs[laux[0]] += int(laux[2])
        else:
            regs[laux[0]] -= int(laux[2])
    return 1


In [7]:
for l in data:
    parse(l)

In [10]:
max(regs.values())

3612

## Challenge 2

In [11]:
regs = defaultdict(lambda: 0)
maxval = 0
for l in data:
    parse(l)
    if max(regs.values()) > maxval:
        maxval = max(regs.values())
maxval

3818